# Hyperparameter Optimization

## Preliminary Steps

### Importing the Preprocessing Pipeline

The preprocessing pipeline developed in previous notebooks is imported from the shared module [`utils/housing_preprocessing.py`](utils/housing_preprocessing.py). We use a low default value for `n_clusters` since we will be tuning this hyperparameter.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

from utils.housing_preprocessing import get_preprocessing_pipeline

preprocessing = get_preprocessing_pipeline(n_clusters=10)  # Low default, will be tuned

In [ ]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestRegressor(random_state=42, n_jobs=1)),
])

### Data Loading

The data loading with stratified train/test split is imported from [`utils/load_california.py`](utils/load_california.py).

In [ ]:
from utils.load_california import load_housing_data

X_train, X_test, y_train, y_test = load_housing_data()

## Relevant Hyperparameters

For the preprocessing pipeline:

| Hyperparameter      | Description                                                 |
|---------------------|-------------------------------------------------------------|
| `n_clusters`        | Number of clusters corresponding to geographic zones.   |
| `gamma`             | Rate of decay for similarity with the centroid.        |
| `strategy`          | Imputation strategy for missing values (default is mean).        |

For RandomForestRegressor:

| Hyperparameter      | Description |
|---------------------|-------------|
| `n_estimators`     | Number of trees in the forest. More trees can improve accuracy but increase computation time. |
| `max_depth`        | Maximum depth of each tree. A low value may lead to *underfitting*, while a high value may lead to *overfitting*. |
| `max_features`     | Number of *features* considered at each split. Can be an integer, a percentage, `"sqrt"` or `"log2"`. Fewer *features* can reduce variance (and thus *overfitting*). |
| `min_samples_split` | Minimum number of samples required to split a node. Higher values reduce *overfitting*. |
| `min_samples_leaf`  | Minimum number of samples in a leaf. Higher values smooth the prediction. |
| `max_samples`      | Percentage of samples used in each tree. Useful for reducing *overfitting*. |

## Hyperparameter Tuning

### 1st Iteration

Let's start with a preliminary randomized search using a broad range of values.

In [ ]:
%%time

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint, uniform

param_dist = {
    'preprocessing__geo__n_clusters': randint(low=3, high=200),
    'random_forest__n_estimators': randint(100, 500),  # Any integer between 100 and 499
    'random_forest__max_depth': randint(10, 110),      # Any integer between 10 and 109
    'random_forest__min_samples_split': randint(2, 20),
    'random_forest__min_samples_leaf': randint(1, 20),
    'random_forest__max_features': ['sqrt', 'log2', None]
}

rnd_search = RandomizedSearchCV(
    estimator = full_pipeline, 
    param_distributions=param_dist, 
    n_iter=40, 
    cv=5,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    n_jobs=-1   # Use all CPU cores in parallel
    )

_ = rnd_search.fit(X_train, y_train)

```%%time``` is a [Jupyter magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time) that measures the execution time of the cell

We can view the results of the best models found:

In [ ]:
cv_res = pd.DataFrame(rnd_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)

cv_res = cv_res[['param_preprocessing__geo__n_clusters',
                 'param_random_forest__n_estimators',
                 'param_random_forest__max_depth',
                 'param_random_forest__min_samples_split',
                 'param_random_forest__min_samples_leaf',
                 'param_random_forest__max_features',
                 "mean_test_score"]]
cv_res.columns = ["n_clusters", "n_estimators", "max_depth", "min_samples_split", "min_samples_leaf", "max_features", "mean_test_score"]

cv_res["mean_test_score"] = -cv_res["mean_test_score"].round().astype(np.int64)
cv_res.head()

Now we can perform successive iterations by fixing *features* where all the best results have converged to a single value, and defining a narrower dictionary of test values centered around the best results.

### 2nd Iteration

In [ ]:
%%time

full_pipeline.set_params(random_forest__max_features="sqrt") # Fix the value of max_features, which has converged to "sqrt"

param_dist = {
    'preprocessing__geo__n_clusters': randint(low=55, high=150),
    'random_forest__n_estimators': randint(200, 300),
    'random_forest__max_depth': randint(44, 97),
    'random_forest__min_samples_split': randint(2, 14),
    'random_forest__min_samples_leaf': randint(1, 5),
}

rnd_search = RandomizedSearchCV(
    estimator = full_pipeline, 
    param_distributions=param_dist, 
    n_iter=40, 
    cv=5,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    n_jobs=-1   # Use all CPU cores in parallel
    )

_ = rnd_search.fit(X_train, y_train)

In [ ]:
cv_res = pd.DataFrame(rnd_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)

cv_res = cv_res[['param_preprocessing__geo__n_clusters',
                 'param_random_forest__n_estimators',
                 'param_random_forest__max_depth',
                 'param_random_forest__min_samples_split',
                 'param_random_forest__min_samples_leaf',
                 "mean_test_score"]]
cv_res.columns = ["n_clusters", "n_estimators", "max_depth", "min_samples_split", "min_samples_leaf", "mean_test_score"]

cv_res["mean_test_score"] = -cv_res["mean_test_score"].round().astype(np.int64)
cv_res.head()